Using models from HF to do a sentiment classification

In [1]:
from transformers import pipeline
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

/home/tian/anaconda3/envs/hf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
classifier = pipeline("sentiment-analysis")

In [ ]:
pred = classifier("Join the growing community on the Hub, forum, or Discord today")
pred

In [ ]:
pred = classifier(["Join the growing community on the Hub, forum, or Discord today", 
"At Microsoft, AI software development is guided by a set of six principles, designed to ensure that AI applications provide amazing solutions to difficult problems without any unintended negative consequences."])
pred

specify model name

In [ ]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
classifier = pipeline("sentiment-analysis", model=model_name)

In [ ]:
pred = classifier(["Join the growing community on the Hub, forum, or Discord today", 
"At Microsoft, AI software development is guided by a set of six principles, designed to ensure that AI applications provide amazing solutions to difficult problems without any unintended negative consequences."])
pred

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
classifier = pipeline("sentiment-analysis", model=model_name)

In [ ]:
pred = classifier(["Join the growing community on the Hub, forum, or Discord today", 
"At Microsoft, AI software development is guided by a set of six principles, designed to ensure that AI applications provide amazing solutions to difficult problems without any unintended negative consequences."])
pred

using tokenizer

In [ ]:
tokens = tokenizer.tokenize("Join the growing community on the Hub, forum, or Discord today")
input_ids = tokenizer("Join the growing community on the Hub, forum, or Discord today")
input_ids

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

In [ ]:
x_train = ["Join the growing community on the Hub, forum, or Discord today", 
"At Microsoft, AI software development is guided by a set of six principles, designed to ensure that AI applications provide amazing solutions to \
difficult problems without any unintended negative consequences."]

batch = tokenizer(x_train, padding=True, truncation=True, max_length=512, return_tensors='pt')  # pt for pytorch

In [ ]:
print(batch)

In [ ]:
with torch.no_grad():
    outputs = model(**batch)

print(outputs)
predictions = F.softmax(outputs.logits, dim=1)
labels = torch.argmax(predictions, dim=1)
print(labels)
labels_text = [model.config.id2label[label_id] for label_id in labels.tolist()]
labels_text

want a loss

In [ ]:
with torch.no_grad():
    outputs = model(**batch, labels=torch.tensor([1, 0]))

print(outputs)
predictions = F.softmax(outputs.logits, dim=1)
labels = torch.argmax(predictions, dim=1)
print(labels)
labels_text = [model.config.id2label[label_id] for label_id in labels.tolist()]
labels_text

save and load

In [ ]:
save_directory = "saved"

In [ ]:
tokenizer.save_pretrained(save_directory)

In [ ]:
model.save_pretrained(save_directory)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_directory)
model = AutoModelForSequenceClassification.from_pretrained(save_directory)

using a German sentiment classification model

In [ ]:
model_name = "oliverguhr/german-sentiment-bert"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
texts = ["Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]

batch = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')  # pt for pytorch
print(batch)

with torch.no_grad():
    outputs = model(**batch)
    # predictions = F.softmax(outputs.logits, dim=1)
    labels = torch.argmax(outputs.logits, dim=1)
    print(labels)
    labels_text = [model.config.id2label[label_id] for label_id in labels.tolist()]
    print(labels_text)

fine-tuning

1. prepare dataset

2. load pretrained tokenizer, call it with dataset -> encoding

3. build pytorch dataset w encoding

4. load pretrained model

5.  a) load trainer and train it

    b) or use native pytorch training pipeline

In [2]:
from torch.utils.data import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from pathlib import Path
from sklearn.model_selection import train_test_split

In [3]:
model_name = "distilbert-base-uncased"

In [4]:
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ['pos', 'neg']:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir == 'neg' else 1)

    return texts, labels

In [5]:
train_texts, train_labels = read_imdb_split('/home/tian/Projects/d2l/data/aclImdb/train')
test_texts, test_labels = read_imdb_split('/home/tian/Projects/d2l/data/aclImdb/test')

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=2)

In [7]:
class IMDbDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, index) :
        item = {key: torch.tensor(val[index])for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[index])
        return item
    
    def __len__(self):
        return len(self.labels)

In [8]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # a generate tokenizer

In [9]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [10]:
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [11]:
training_arguments = TrainingArguments(
    output_dir='./results',               # output directory
    num_train_epochs=2,                   # max traning epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size per device during evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    learning_rate=5e-5,                   # lr
    weight_decay=0.01,
    logging_dir='./logs',                 # directory for storing logs
    logging_steps=10    
)

In [14]:
model = DistilBertForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

In [15]:
trainer.train()

/home/tian/anaconda3/envs/hf/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24998
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3126
  Number of trainable parameters = 66955010
  0%|          | 10/3126 [00:03<16:05,  3.23it/s]

{'loss': 0.6983, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  1%|          | 20/3126 [00:06<15:41,  3.30it/s]

{'loss': 0.6979, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}


  1%|          | 30/3126 [00:09<15:39,  3.29it/s]

...

100%|█████████▉| 3120/3126 [16:03<00:01,  3.26it/s]

{'loss': 0.0889, 'learning_rate': 1.1424219345011426e-07, 'epoch': 2.0}


100%|██████████| 3126/3126 [16:05<00:00,  3.92it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 3126/3126 [16:05<00:00,  3.24it/s]

{'train_runtime': 965.6626, 'train_samples_per_second': 51.774, 'train_steps_per_second': 3.237, 'train_loss': 0.222401846269347, 'epoch': 2.0}


TrainOutput(global_step=3126, training_loss=0.222401846269347, metrics={'train_runtime': 965.6626, 'train_samples_per_second': 51.774, 'train_steps_per_second': 3.237, 'train_loss': 0.222401846269347, 'epoch': 2.0})

fine-tuning w classic pytorch training loop

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

In [ ]:
device = torch.device('cuda')

In [ ]:
model2 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model2.to(device)

In [ ]:
model2.train()

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_train_epochs = 2

In [ ]:
for epoch in range(num_train_epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model2(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs[0]
        loss.backward()
        optimizer.step()

Inference

In [ ]:
model.eval()